## Code for Supplementary Figure 4

In [ ]:
# using kernel .mne-python (Python 3.10.10)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import pandas as pd
from scipy.stats import permutation_test

from pathlib import Path
from scipy.stats import (
    ttest_ind, bootstrap
)
from scipy import io
import statsmodels.api as sm

import mne
from mne.viz.topomap import _prepare_topomap_plot, _make_head_outlines

import h5io
import pyriemann

from meeglet import define_frequencies

from mpl_toolkits.axes_grid1 import ImageGrid

 Create adjacency matrix

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = (
    str(sample_data_folder) + "/MEG/sample/sample_audvis_filt-0-40_raw.fif"
)
raw_for_adjacency = mne.io.read_raw_fif(sample_data_raw_file)
meg_indices = raw_for_adjacency.pick_types(meg='mag')
adj_matrix = mne.channels.find_ch_adjacency(raw_for_adjacency.info, 'mag')[0]

In [ ]:
foi = define_frequencies(foi_start=1, foi_end=64, bw_oct=0.35, delta_oct=0.05)[0]

### IMPORT FEATURES

In [ ]:
features_CBU = h5io.read_hdf5('./BIOFIND/cosmeeg_CBU_2023-06-22_10-06.h5')
features_CTB1 = h5io.read_hdf5('./BIOFIND/cosmeeg_CTB_2023-06-22_10-46.h5')
features_CTB2 = h5io.read_hdf5('./BIOFIND/cosmeeg_CTB_2023-06-23_11-04.h5')

In [ ]:
features_all = features_CTB1 | features_CTB2 | features_CBU #merge 3 hdf5 files (key: patient, value : tuple cov, pow)

In [ ]:
features_all.keys()
features_all['Sub0014'][1].shape

In [ ]:
pow = np.array([features_all[subject][1] for subject in features_all]) # 0 cov, 1 pow, 2 csd
pow.shape

In [ ]:
participants_fname = './BIOFIND/participants.tsv'
subject_df = pd.read_csv(participants_fname, delimiter='\t')
subject_df['participant_id'] = subject_df['participant_id'].str.replace('sub-', '')
subject_df = subject_df.set_index('participant_id')

### Build subject groups

In [ ]:
mean_mmse = subject_df['MMSE'].mean()
subject_df['MMSE'] = subject_df['MMSE'].fillna(mean_mmse)
mean_edu = subject_df['Edu_years'].mean()
subject_df['Edu_years'] = subject_df['Edu_years'].fillna(mean_edu)

In [ ]:
subject_df = subject_df.loc[features_all.keys()]
subject_df.head()

# Analysis PSD


In [ ]:
feature_mmse = list(subject_df['MMSE'])
feature_control = [ 1 if is_control else 0 for is_control in subject_df['group'] == 'control']
feature_converter = subject_df['Converters'].fillna(-1)

In [ ]:
psds = np.array([features_all[subject][1] for subject in features_all]) # 0 cov, 1 pow, 2 csd
psds = 10*np.log10(psds) #dB
psds_mean_chan = np.mean(psds, 1) #mean over channels
psds_mean_chan.shape

In [ ]:
del features_all, features_CBU, features_CTB1, features_CTB2

## ADJUST FOR CONVERTERS

In [ ]:
subject_df_adjust3 = subject_df.copy()
subject_df_adjust3['Converters'] = subject_df_adjust3['Converters'].fillna(-1)
subject_df_adjust3.loc[subject_df_adjust3['Converters'] >= 0].index.to_numpy()
converters_idx = subject_df_adjust3['Converters'] >= 0
psds_mean_chan[converters_idx].shape

In [ ]:
# adjust on all confounds: site, sex, age, Recording_time, Edu_years, Move1, Move2, Pre_task
subject_df_adjust = subject_df[['site']] #subject_df[['site','sex','age', 'Recording_time','Edu_years','Move1','Move2','Pre_task']].copy() # select specific columns 
for feature in ['site']: # for categorical features > generate 2 columns for 'site' and 'sex'
    dummies = pd.get_dummies(subject_df[feature], drop_first = True) #drop_first =True ?
    subject_df_adjust = pd.concat([subject_df_adjust.drop(columns=feature), dummies], axis=1) #integrate new columns in subjects_df_adjust
#subject_df_adjust.head()

z = subject_df_adjust.to_numpy()[converters_idx]

psds_clean_converter = np.zeros_like(psds[converters_idx])

for chan in range(psds.shape[1]):
    for freq in range(psds.shape[2]):

        lm = LinearRegression() #initialize the model
        X = psds[converters_idx, chan, freq]

        lm = lm.fit(z, X) 
        X_clean = X - lm.predict(z) + lm.intercept_
        
        psds_clean_converter[:, chan, freq] = X_clean

psds_clean_mean_converter = np.mean(psds_clean_converter, 1)

In [ ]:
psds_clean_mean_converter.shape

## PLOT ADJUSTED PSD CONVERTERS 

In [ ]:
plt.rcParams['font.size'] = 11
fig, axes = plt.subplots(1,1, figsize=[3.6,1.4],sharey=True)
psd_mean_converter = np.mean(psds_clean_mean_converter[np.array(feature_converter[feature_converter >= 0]) == 1],0)
psd_mean_non_converter = np.mean(psds_clean_mean_converter[np.array(feature_converter[feature_converter >= 0]) == 0],0)

plt.plot(foi, psd_mean_converter, label='AD progression', linewidth = 2.2)
plt.plot(foi, psd_mean_non_converter, label = 'Stable MCI', linewidth = 2.2)

legend = plt.legend(loc='upper right', bbox_to_anchor=(0.6, 0.43))
frame = legend.get_frame()
frame.set_linewidth(0)  
plt.title('Average power spectra', fontsize = 11, y=1)
plt.xlabel('Frequencies (Hz)')
plt.ylabel('Power (dB)')
plt.ylim(-280,-260)
plt.xscale('log', base =2)
plt.xticks([1,2,4,8,16,32,64],labels = [1,2,4,8,16,32,64])
#plt.xticks(plt.xticks()[0][2:-2], labels = [1,2,4,8,16,32,64])
sns.despine(offset=10, trim=True);[converters_idx]
#plt.show()
plt.savefig('./figures/average_meg_power_site.png', bbox_inches='tight')

## T-test to compare mean PSD between groups (converters vs non converters) at definite frequencies

In [ ]:
# T test to compare mean psd between groups at definite frequencies (2, 4, 8, 16, 32Hz)
psds_mean_channels_converters_all = np.mean((psds_clean_converter[:,:,20:120:20]),1)
psds_mean_channels_converters = psds_mean_channels_converters_all[np.array(feature_converter[converters_idx]) == 1]
psds_mean_channels_nonconverters = psds_mean_channels_converters_all[np.array(feature_converter[converters_idx]) == 0]

t_statistic, p_value_conv = ttest_ind(psds_mean_channels_converters, psds_mean_channels_nonconverters, axis=0, equal_var=False)
print(p_value_conv*5)

In [ ]:
psdsT = psds_clean_converter.transpose(0,2,1) # converter/non

In [ ]:
psds_mean_channels_converters_all.shape
np.mean(psdsT[feature_converter[converters_idx] == 0],2).shape

psds_clean_mean_converter[np.array(feature_converter[feature_converter >= 0]) == 1].shape
psds_clean_mean_converter[np.array(feature_converter[feature_converter >= 0]) == 0].shape

## Permutation F-test on sensor data averaged over all sensors

In [ ]:
data_condition_1 = psds_clean_mean_converter[np.array(feature_converter[feature_converter >= 0]) == 0]
data_condition_2 = psds_clean_mean_converter[np.array(feature_converter[feature_converter >= 0]) == 1]

# Prepare your data for cluster-based permutation test
X = [data_condition_1, data_condition_2]

In [ ]:
data_condition_2.shape

## TFCE F-test method to compare averaged metrics between groups


In [ ]:
np.mean(psdsT[feature_converter[converters_idx] == 1],2).shape 

In [ ]:
T_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test(
    [data_condition_2,data_condition_1],
    threshold=dict(start=0, step=0.2),
    n_jobs=None, 
    n_permutations = 10000, 
    tail=0, out_type="mask"
)

In [ ]:
print(cluster_p_values)

### Vizualization

In [ ]:
rng = np.random.RandomState(23)

def my_statistic(sample1, sample2, axis=0):
    statistic = np.mean(sample1) - np.mean(sample2)
    return statistic

condition1=data_condition_1
condition2=data_condition_2

results = list()

for ii in range(condition2.shape[1]):
    data = (condition2[:, ii], condition1[:, ii])
    res = bootstrap(data, my_statistic, method='basic', random_state=rng, vectorized=False)
    results.append(res)    

conf_ints = np.array([tuple(res.confidence_interval) for res in results])
conf_ints.shape                 

In [ ]:
plt.rcParams['font.size'] = 11

# mean PSD difference
mean_difference = condition2.mean(axis=0) - condition1.mean(axis=0)

# IC95 for the difference
upper_bound = conf_ints[:,1]
lower_bound = conf_ints[:,0]

fig, ax = plt.subplots(1, 1, figsize=[3.6,1.4], sharey=True)
ax.set_title('PSD difference between groups', fontsize = 11.5, y=1.05)
ax.set_xscale('log', base=2)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
xticks = [1, 2, 4, 8, 16, 32, 64]
ax.set_xticks(xticks)
ax.set_xticklabels(xticks)
ax.set_xlim(1, 64)
ax.axhline(0, color='black', linestyle= "--")
sns.despine(offset=5, trim=False)

# Plot the mean difference
line = ax.plot(
    foi,
    mean_difference,
    label="Mean Difference"
)

# Fill the area between upper and lower CI bounds
ax.fill_between(foi,upper_bound, lower_bound, alpha=0.5, label="95% CI")

ax.set_ylabel("PSD difference (dB)")
ax.set_xlabel("Frequencies (Hz)")

plt.savefig('./figures/suppl_figure4b_site.pdf', bbox_inches='tight')


In [ ]:
# Assuming `foi`, `T_obs`, and `cluster_p_values` are defined
mask = cluster_p_values <= 0.10

plt.rcParams['font.size'] = 11
fig, ax2 = plt.subplots(1, 1, figsize=[3.6, 1.4], sharey=True)
ax2.set_title('Permutation F-test on mean PSD', fontsize = 11.5, y=1.05)

# Plot the main black line
hf, = ax2.plot(foi, T_obs, "black")

# Find contiguous regions of significant frequencies
inside_cluster = False
fill_plotted = False  # Flag to track if the green fill has already been labeled

for i in range(len(foi)):
    if mask[i] and not inside_cluster:
        start_idx = i
        inside_cluster = True
    elif not mask[i] and inside_cluster:
        end_idx = i
        # Fill between for this cluster region
        if not fill_plotted:
            ax2.fill_between(foi[start_idx:end_idx], y1=T_obs[start_idx:end_idx], y2=0, 
                             color="green", label='cluster P < 0.10', alpha=0.3)
            fill_plotted = True  # Set flag to True so that the label is only used once
        else:
            ax2.fill_between(foi[start_idx:end_idx], y1=T_obs[start_idx:end_idx], y2=0, 
                             color="green", alpha=0.3)
        inside_cluster = False
# If a cluster goes till the end of the data
if inside_cluster:
    if not fill_plotted:
        ax2.fill_between(foi[start_idx:], y1=T_obs[start_idx:], y2=0, color="green", alpha=0.3, label='P < 0.10')
    else:
        ax2.fill_between(foi[start_idx:], y1=T_obs[start_idx:], y2=0, color="green", alpha=0.3)

# Set axes labels and scales
ax2.set_xlabel("Frequencies (Hz)")
ax2.set_ylabel("F-values")
ax2.set_xscale('log', base=2)
xticks = [1, 2, 4, 8, 16, 32, 64]
ax2.set_xticks(xticks)
ax2.set_xlim(1, 64)
ax2.set_ylim(0,1.5)
ax2.set_xticklabels(xticks)

# Customize appearance and remove unnecessary spines
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
sns.despine(offset=5, trim=False)

# Add the legend
ax2.legend(loc='upper right', bbox_to_anchor=(0.58, 0.8), ncol=2, fontsize=10, frameon=False)


# Save the figure
plt.savefig('./figures/suppl_figure4c_site_p<0.1.pdf', bbox_inches='tight')


In [ ]:
# print significant frequencies for clusters
print(foi[mask])

### Non parametric two-tailed permutation test (uncorrected for multiple comparisons) to find frequencies with p-val < 0.05 

In [ ]:
rng = np.random.RandomState(23)

# Define custom statistic function: difference of means
def my_statistic(sample1, sample2, axis=0):
    return np.mean(sample1, axis=axis) - np.mean(sample2, axis=axis)

# Generate or define condition1 and condition2 data arrays
condition1 = np.mean(psdsT[feature_converter[converters_idx] == 0], axis=2)
condition2 = np.mean(psdsT[feature_converter[converters_idx] == 1], axis=2)

# Initialize list to store indices of features with significant p-values
significant_indices = []

# Loop through each feature to perform permutation test
for i in range(condition1.shape[1]):
    # Extract data for the current feature from both conditions
    data = (condition1[:, i], condition2[:, i])
    
    # Perform permutation test
    res = permutation_test(data, my_statistic, vectorized=True, random_state=rng)
    
    # If p-value < 0.05, add the feature index to the list of significant features
    if res.pvalue < 0.05:
        significant_indices.append(i)

print("Indices with p-value below 0.05:", significant_indices)

In [ ]:
foi[significant_indices]

### CLUSTER BASED PERMUTATION TEST TO COMPARE TOPOMAPS BETWEEN GROUPS (keeping spatial info on 102 channels)

In [ ]:
psds_converters = psds_clean_mean_converter[feature_converter[converters_idx] == 1] # psd array adjusted for converters
psds_nonconverters = psds_clean_mean_converter[feature_converter[converters_idx] == 0]

In [ ]:
psds_clean_mean_converter.shape

In [ ]:
psdsT = psds_clean_converter.transpose(0,2,1) # converter/non

In [ ]:
psdsT[feature_converter[converters_idx] == 0].shape

In [ ]:
# cluster permutation test with TFCE keeping channel info and frequencies info
F, c, alpha, h = mne.stats.spatio_temporal_cluster_test([
    psdsT[feature_converter[converters_idx] == 0],
    psdsT[feature_converter[converters_idx] == 1]],
    threshold=dict(start=0, step=0.2),
    n_jobs=4, n_permutations=1000, tail=0,
    adjacency=adj_matrix)
print(alpha)

In [ ]:
# c has shape 29*102=2958 (tuple : (freq, chan)), each tuple will be called cl
# alpha has shape 29*102=2958,containing pvalues
p_clust = np.zeros((psdsT.shape[1],psdsT.shape[2])) # init p_clust, shape freq, channels
for cl, p in zip(c, alpha): # fills in p_clust array 
    p_clust[cl]=-np.log10(p)
p_clust.shape
mask = np.ones_like(p_clust, dtype = bool)
#mask[p_clust<1.3] = False  for p<0.05
mask[p_clust<1] = False # for p<0.10

# will enable to create topomaps of p values for each frequency for each channel

In [ ]:
foi[79]

In [ ]:
idx = np.where(mask.sum(axis=1)>1)[0]  # indique les frequences avec plus de 5 electrodes en clusters significatifs (index des frequences) 
foi[idx]
print(foi[idx[::6]]) # utilise un pas de 6

In [ ]:
psds_converters = psds_clean_converter[feature_converter[converters_idx] == 1]
psds_nonconverters = psds_clean_converter[feature_converter[converters_idx] == 0]
psds_converters_mean = np.mean(psds_converters, 0)
psds_nonconverters_mean = np.mean(psds_nonconverters, 0)

## PLOT TOPOMAPS FOR CONVERTERS VS NON CONVERTERS DIFF

## Plot function

In [ ]:
def topoplot(    
    data,  # 1D array or list with values corresponding to the channels in channel_names
    channel_names, # list of channel names, e.g. ["Fp1", "Fp2", ...]
    info,  
    montage = "standard_1020",
    cmap='RdBu_r',
    scale_limits=(None, None),  # useful to plot several topographies on the same scale
    size=1,  # size of the topoplot
    axes=None,
    mask=None,
):
    assert len(data) == len(channel_names)

    _, pos, _, _, ch_type, sphere_, clip_origin = _prepare_topomap_plot(
        info,
        "mag",
        sphere=(0, 0.023, 0.021, 0.1)
    )
    
    outlines_ = _make_head_outlines(sphere_, pos, 'head', clip_origin)
    
    im, cn = mne.viz.plot_topomap(
        data=data, 
        pos=pos, 
        axes=axes,
        vlim = scale_limits,
        cmap=cmap,
        outlines=outlines_,
        image_interp='cubic',
        size=size,
        mask=mask,
        show=False,
    )

    return im, cn

def plot_topoplot_grid(
    data, # data[i][j] contains a 1D list or array with data for the topo plot in row i and col j
    info, 
    row_labels,  # list of row labels that describe the rows in data, e.g. ["Condition 1", "Condition 2"]
    col_labels,  # list of col labels that describe the cols in data, e.g. ["Alpha" ,"Beta", "Gamma"]
    channel_names,  # channel labels in corresponding order to the 1D lists or arrays in data
    cbar_mode="single",  # "single" for shared scale, "each" for individual scales
    cbar_label=r"$10\times\log_{10}fT^2/Hz$[dB]",  # label for the color bar,
    #cbar_label=r"dB",
    cmap='RdBu_r',
    scale_limits=(None, None),
    mask=None,
):
    # Figure Grid Params
    x_label_size=12
    y_label_size=12
    x_label_pad=10
    y_label_pad=10
    
    cbar_fmt='%3.2f'
    cbar_size="5%"
    cbar_pad=0.1 if cbar_mode == "each" else 0.8
    axes_pad=(0.9 if cbar_mode == "each" else 0.4 , 0.4)
    clabel_size=10
    fig_size=(12, 4)
    rect=(0.05, 0.05, 0.90, 0.95)

    nrows = len(data)
    ncols = len(data[0])

    fig = plt.figure(figsize=fig_size)
    grid = ImageGrid(
        fig, rect,
        nrows_ncols=(nrows, ncols),
        axes_pad=axes_pad, share_all=True, cbar_location="right",
        cbar_mode=cbar_mode, cbar_size=cbar_size, cbar_pad=cbar_pad,
    )

    
    if cbar_mode == "single" and scale_limits == (None, None):
        scale_limits = np.array(data).min(), np.array(data).max()
    elif cbar_mode == "each" and scale_limits == "columns":
        mins = np.min(np.hstack(data), axis=1)
        maxs = np.max(np.hstack(data), axis=1)
 
    scale_limits_ = scale_limits
    axes = grid.axes_row
    for row, row_label in enumerate(row_labels):
        for col, col_label in enumerate(col_labels):
            ax = axes[row][col]
            if cbar_mode == "each" and scale_limits == "columns":
                scale_limits_ = [mins[col], maxs[col]]
            im, _ = topoplot(data[row][col], channel_names, info, scale_limits=scale_limits_,
                             axes=ax,
                             cmap=cmap,
                             mask= None if mask is None else mask[row][col])

            ax.set_xlabel(col_label, fontsize=x_label_size, labelpad=x_label_pad)
            ax.set_ylabel(row_label, fontsize=y_label_size, labelpad=y_label_pad)

            cbar = ax.cax.colorbar(im)
            cbar.ax.set_ylabel(cbar_label, fontsize=clabel_size)
            ax.cax.toggle_label(True)
     
    return fig

In [ ]:
def min_max(c):
    this_min, this_max = np.min(c), np.max(c)
    my_max = max(abs(this_min), abs(this_max))
    return my_max

In [ ]:
difference = np.squeeze(np.array([psds_converters_mean - psds_nonconverters_mean]))

data_diff = [
    [difference[:, ix] for ix in idx[::4][:5]],
]
mask_grid = [
    [mask.T[:, ix] for ix in idx[::4][:5]],
]
row_labels = ["Difference"]
col_labels = [f"{f:0.1f} Hz" for f in foi[idx[::4][:5]]]
channel_names = raw_for_adjacency.info.ch_names

fig = plot_topoplot_grid(data_diff, raw_for_adjacency.info, row_labels, col_labels, channel_names, scale_limits = [lambda x: -min_max(x), min_max],
                         cbar_mode="single", mask=mask_grid,
                         );
fig.set_size_inches(10, 2)
plt.savefig('./figures/suppl_figure4_topoplot_diff_top_site_p<0.10.pdf', bbox_inches='tight')